# How to Factor Quadratic Polynomials

This is a step-by-step guide to factoring any 3-term polynomial that can, in principle, be factored. To start, suppose we have a polynomial that looks like:

$$ax^2 + bx + c$$

In a real example, the terms for $a$, $b$, and $c$ will be integer constants, but we're using variables here to show that this technique is general.  Our goal is to find two simpler expressions whose product is equal to the polynomial.  In __every__ case, if there is an answer then it will take the form of:

$$(rx+s)(tx+u)$$

with $r$, $s$, $t$, $u$ being integers.  Thus, we're looking for specific values for those integers such that:

$$(rx+s)(tx+u) = rtx^2 + (ru + st)x + su = ax^2 + ax + c$$

This guide will do two things.  First, it will randomly generate a random quadratic factoring problem (that you can rerun to get a new problem over-and-over again).  Second, it will solve the problem in a step-by-step manner illustrating how to find the factors of the polynomial.

Each part of this guide is written as a small bit of python code, but we'll have commentary at each step.

---

## 1. Generate a random example

In [179]:
# Need to easily get random numbers.
import random

# Produce a non-zero random number between -max ... max.
def random_digit(max_size):
    # Get a number between 1 ... max.
    num = random.randint(1, max_size)
    # Flip a coin to determine the sign.
    sign = 1 if random.random() < 0.5 else -1
    return num * sign

In [180]:
# Produce a random quadratic polynomial that can be factored.

def generate_polynomial():
    # We will restrict the problems so that their solution always have small integers.
    max_size = 5
    r = random_digit(max_size)
    s = random_digit(max_size)
    t = random_digit(max_size)
    u = random_digit(max_size)
    # The answer will be (rx+s)(tx+u) but we're keeping that a secret for now.
    # Instead, we cross multiply all of the terms to produce ax^2 + ax + c, as in:
    # (rx+s)(tx+u) = rt x^2 + (ru + st) x + su = ax^2 + bx + c
    a = r * t
    b = r * u + s * t
    c = s * u
    # We only need to return the three coefficients
    return (a, b, c)

In [181]:
# Convert the three coefficients into a string that prints how we would expect.
# The only reason for this function is to help us pretty print quadratics.

def polynomial_string(abc):
    (a, b, c) = abc
    result = str(a) if abs(a) > 1 else ""
    result += "-" if a == -1 else ""
    result += "x^2" if abs(a) >= 1 else ""
    if b:
        if a: 
            result += " - " if b < 0 else " + "
            result += str(abs(b)) if abs(b) > 1 else ""
            result += "x"
        else:
            result += "-" if b < 0 else ""
            result += str(abs(b)) if abs(b) > 1 else ""
            result += "x"
    if c:
        result += " - " if c < 0 else " + "
        result += str(abs(c))
    return result


The equation below is the quadratic that we want to factor.

In [182]:
(a, b, c) = generate_polynomial()
print("(a, b, c) =", (a, b, c))
print(polynomial_string((a, b, c)))

(a, b, c) = (6, -7, -20)
6x^2 - 7x - 20


---

## 2. Find the groups

We'll now find a way of splitting the middle term of the polynomial into two parts.  First, multiply a * c.

In [183]:
ac = a * c
print(ac)

-120


We're now going to search over all factors of $ac$ (i.e., all possible pairs of integers, $j$ and $k$, such that $jk = ac$).
If we find two factors that add up to $b$ (the middle term in the quadratic), then those will tell us how to form the groups.

In [184]:
# At the end of this code block, j and k will contain the numbers that we need. 
j, k = None, None
for i in range(1, abs(ac) + 1):
    if ac % i == 0:
        j = -i if (a < 0 and b < 0) or (a > 0 and b < 0) else i
        k = ac // j
        status = "SUCCESS!" if (j + k == b) else "\t"
        print ("%s\t%4d  * %4d = %4d \t %4d  + %4d = %4d" % (status, j, k, j*k, j, k, j+k))
        if (j + k == b): break

		  -1  *  120 = -120 	   -1  +  120 =  119
		  -2  *   60 = -120 	   -2  +   60 =   58
		  -3  *   40 = -120 	   -3  +   40 =   37
		  -4  *   30 = -120 	   -4  +   30 =   26
		  -5  *   24 = -120 	   -5  +   24 =   19
		  -6  *   20 = -120 	   -6  +   20 =   14
		  -8  *   15 = -120 	   -8  +   15 =    7
		 -10  *   12 = -120 	  -10  +   12 =    2
		 -12  *   10 = -120 	  -12  +   10 =   -2
SUCCESS!	 -15  *    8 = -120 	  -15  +    8 =   -7


We're now going to split the quadratic into four terms.  Remember, we started with:

In [185]:
print(polynomial_string((a, b, c)))

6x^2 - 7x - 20


We can convert the above into $(ax^2 + jx) + (kx + x)$.

In [186]:
LHS = polynomial_string((a, j, 0))
RHS = polynomial_string((0, k, c))
print('(', LHS, ") + (", RHS, ')', sep='')

(6x^2 - 15x) + (8x - 20)


Now, we can factor the above in parts by computing the greatest common denominators for the terms on the left and right sides (seperately).

In [187]:
# Euclid's algorithm for finding the GCD

def gcd(x, y):
    return x if y == 0 else gcd(y, x % y) 

We'll refer to the GCD of the left and right sides as l and r


In [188]:
l = gcd(a, j)
r = gcd(k, c)

These newly factored groups can now be written as:

In [189]:
LHS = str(l) + "x * (" + polynomial_string((0, a // l, j // l)) + ")"
RHS = str(r) + " * (" +  polynomial_string((0, k // r, c // r)) + ")"
print(LHS, "+", RHS)

-3x * (-2x + 5) + -4 * (-2x + 5)


If we did everything correctly, then then the refactored groups should now be the same, allowing us to treat the group as a common factor.

In [190]:
LHS = polynomial_string((0, l, r))
RHS = polynomial_string((0, a // l, j // l))
print(polynomial_string((a, b, c)), "  =  ", "(" + LHS + ") * (" + RHS + ")")


6x^2 - 7x - 20   =   (-3x - 4) * (-2x + 5)
